In [1]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/University (M.S., Stanford)/CS 229S - Systems for ML/cs229s-nanoGPT-rmg

/content/drive/MyDrive/University (M.S., Stanford)/CS 229S - Systems for ML/cs229s-nanoGPT-rmg


In [3]:
# !pip install torch numpy transformers datasets tiktoken wandb tqdm memory-profiler torcheval

In [4]:
# !python data/wikitext/prepare.py

In [3]:
import os
import time
import math
import pickle
from contextlib import nullcontext

import numpy as np
import torch

from model import GPTConfig, GPT
from pruning import convert_to_prunable, compress_layers, PrunableLinear

# -----------------------------------------------------------------------------
# default config values designed to train a gpt2 (124M) on OpenWebText
# I/O
out_dir = 'out'
eval_interval = 50  # TODO - 100
log_interval = 1
eval_iters = 200
eval_only = False # if True, script exits right after the first eval
always_save_checkpoint = False # if True, always save a checkpoint after each eval
init_from = 'resume' # 'gpt2-medium' # 'scratch' or 'resume' or 'gpt2*'
# wandb logging
wandb_log = False # disabled by default
wandb_project = 'cs229s'
wandb_run_name = 'gpt2' # 'run' + str(time.time())
# data
dataset = 'wikitext'
gradient_accumulation_steps = 5 * 8 # used to simulate larger batch sizes
batch_size = 4 # if gradient_accumulation_steps > 1, this is the micro-batch size
block_size = 1024
# model
n_layer = 12
n_head = 12
n_embd = 768
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
bias = False # do we use bias inside LayerNorm and Linear layers?
# adamw optimizer
learning_rate = 6e-4 # max learning rate
max_iters = 50 # TODO - 100 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9
beta2 = 0.95
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
# DDP settings
backend = 'nccl' # 'nccl', 'gloo', etc.
# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
# -----------------------------------------------------------------------------
config_keys = [k for k,v in globals().items() if not k.startswith('_') and isinstance(v, (int, float, bool, str))]
config = {k: globals()[k] for k in config_keys} # will be useful for logging
# -----------------------------------------------------------------------------


# if not ddp, we are running on a single gpu, and one process
seed_offset = 0
tokens_per_iter = gradient_accumulation_steps * batch_size * block_size
print(f"tokens per iteration will be: {tokens_per_iter:,}")

os.makedirs(out_dir, exist_ok=True)
torch.manual_seed(1337 + seed_offset)
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

# poor man's data loader
data_dir = os.path.join('data', dataset)
train_data = np.memmap(os.path.join(data_dir, 'train.bin'), dtype=np.uint16, mode='r')
val_data = np.memmap(os.path.join(data_dir, 'val.bin'), dtype=np.uint16, mode='r')
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    if device_type == 'cuda':
        # pin arrays x,y, which allows us to move them to GPU asynchronously (non_blocking=True)
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y

# init these up here, can override if init_from='resume' (i.e. from a checkpoint)
iter_num = 0
best_val_loss = 1e9

# attempt to derive vocab_size from the dataset
meta_path = os.path.join(data_dir, 'meta.pkl')
meta_vocab_size = None
if os.path.exists(meta_path):
    with open(meta_path, 'rb') as f:
        meta = pickle.load(f)
    meta_vocab_size = meta['vocab_size']
    print(f"found vocab_size = {meta_vocab_size} (inside {meta_path})")

# model init
model_args = dict(n_layer=n_layer, n_head=n_head, n_embd=n_embd, block_size=block_size,
                  bias=bias, vocab_size=None, dropout=dropout) # start with model_args from command line
if init_from == 'scratch':
    # init a new model from scratch
    print("Initializing a new model from scratch")
    # determine the vocab size we'll use for from-scratch training
    if meta_vocab_size is None:
        print("defaulting to vocab_size of GPT-2 to 50304 (50257 rounded up for efficiency)")
    model_args['vocab_size'] = meta_vocab_size if meta_vocab_size is not None else 50304
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
elif init_from == 'resume':
    print(f"Resuming training from {out_dir}")
    # resume training from a checkpoint.
    ckpt_path = os.path.join(out_dir, 'ckpt_150_pruned_0.05_direct.pt')
    checkpoint = torch.load(ckpt_path, map_location=device)
    checkpoint_model_args = checkpoint['model_args']
    # force these config attributes to be equal otherwise we can't even resume training
    # the rest of the attributes (e.g. dropout) can stay as desired from command line
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = checkpoint_model_args[k]
    # create the model
    gptconf = GPTConfig(**model_args)
    model = GPT(gptconf)
    convert_to_prunable(model, device=device_type)
    state_dict = checkpoint['model']
    # fix the keys of the state dictionary :(
    # honestly no idea how checkpoints sometimes get this prefix, have to debug more
    unwanted_prefix = '_orig_mod.'
    for k,v in list(state_dict.items()):
        if k.startswith(unwanted_prefix):
            state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
    model.load_state_dict(state_dict)
    iter_num = checkpoint['iter_num']
    best_val_loss = checkpoint['best_val_loss']
elif init_from.startswith('gpt2'):
    print(f"Initializing from OpenAI GPT-2 weights: {init_from}")
    # initialize from OpenAI GPT-2 weights
    override_args = dict(dropout=dropout)
    model = GPT.from_pretrained(init_from, override_args)
    # read off the created config params, so we can store them into checkpoint correctly
    for k in ['n_layer', 'n_head', 'n_embd', 'block_size', 'bias', 'vocab_size']:
        model_args[k] = getattr(model.config, k)
# crop down the model block size if desired, using model surgery
# model = torch.load('test.pt')
if block_size < model.config.block_size:
    model.crop_block_size(block_size)
    model_args['block_size'] = block_size # so that the checkpoint will have the right value
model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2), device_type)
if init_from == 'resume':
    optimizer.load_state_dict(checkpoint['optimizer'])
checkpoint = None # free up memory

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            with ctx:
                logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

# logging
if wandb_log:
    import wandb
    wandb.init(project=wandb_project, name=wandb_run_name, config=config)

del state_dict, checkpoint
torch.cuda.empty_cache()

tokens per iteration will be: 163,840
Resuming training from out
number of parameters: 353.77M
num decayed parameter tensors: 98, with 354,501,632 parameters
num non-decayed parameter tensors: 194, with 321,536 parameters
using fused AdamW: True


In [4]:
torch.cuda.empty_cache()

In [5]:
def finetune(model, iter_num, best_val_loss, eval_only=False):
  # training loop
  X, Y = get_batch('train') # fetch the very first batch
  t0 = time.time()
  running_mfu = -1.0
  for local_iter_num in range(max_iters):

      # determine and set the learning rate for this iteration
      lr = get_lr(iter_num) if decay_lr else learning_rate
      for param_group in optimizer.param_groups:
          param_group['lr'] = lr

      # evaluate the loss on train/val sets and write checkpoints
      # if iter_num % eval_interval == 0:
      #     losses = estimate_loss()
      #     print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
      #     if wandb_log:
      #         wandb.log({
      #             "iter": iter_num,
      #             "train/loss": losses['train'],
      #             "val/loss": losses['val'],
      #             "lr": lr,
      #             "mfu": running_mfu*100, # convert to percentage
      #         })
      #     if losses['val'] < best_val_loss or always_save_checkpoint:
      #         best_val_loss = losses['val']
      #         if iter_num > 0:
      #             checkpoint = {
      #                 'model': model.state_dict(),
      #                 'optimizer': optimizer.state_dict(),
      #                 'model_args': model_args,
      #                 'iter_num': iter_num,
      #                 'best_val_loss': best_val_loss,
      #                 'config': config,
      #             }
      #             print(f"saving checkpoint to {out_dir}")
      #             torch.save(checkpoint, os.path.join(out_dir, 'ckpt.pt'))
      # if local_iter_num == 0 and eval_only:
      #     break

      # forward backward update, with optional gradient accumulation to simulate larger batch size
      # and using the GradScaler if data type is float16
      for micro_step in range(gradient_accumulation_steps):
          with ctx:
              logits, loss = model(X, Y)
              loss = loss / gradient_accumulation_steps # scale the loss to account for gradient accumulation
          # immediately async prefetch next batch while model is doing the forward pass on the GPU
          X, Y = get_batch('train')
          # backward pass, with gradient scaling if training in fp16
          scaler.scale(loss).backward()
      # clip the gradient
      if grad_clip != 0.0:
          scaler.unscale_(optimizer)
          torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
      # step the optimizer and scaler if training in fp16
      scaler.step(optimizer)
      scaler.update()
      # flush the gradients as soon as we can, no need for this memory anymore
      optimizer.zero_grad(set_to_none=True)

      # timing and logging
      t1 = time.time()
      dt = t1 - t0
      t0 = t1
      if iter_num % log_interval == 0:
          # get loss as float. note: this is a CPU-GPU sync point
          # scale up to undo the division above, approximating the true total loss (exact would have been a sum)
          lossf = loss.item() * gradient_accumulation_steps
          if local_iter_num >= 5: # let the training loop settle a bit
              mfu = model.estimate_mfu(batch_size * gradient_accumulation_steps, dt)
              running_mfu = mfu if running_mfu == -1.0 else 0.9*running_mfu + 0.1*mfu
          print(f"iter {iter_num}: loss {lossf:.4f}, time {dt*1000:.2f}ms, mfu {running_mfu*100:.2f}%")

      iter_num += 1
  return iter_num, best_val_loss

In [ ]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 0: train loss 3.7232, val loss 3.7326
iter 0: loss 3.8476, time 138304.75ms, mfu -100.00%
iter 1: loss 3.5523, time 31620.78ms, mfu -100.00%
iter 2: loss 3.8951, time 31535.70ms, mfu -100.00%
iter 3: loss 3.6432, time 31759.12ms, mfu -100.00%
iter 4: loss 3.6798, time 31835.23ms, mfu -100.00%
iter 5: loss 3.7793, time 31738.59ms, mfu 4.01%
iter 6: loss 3.6765, time 31699.78ms, mfu 4.01%
iter 7: loss 3.6175, time 31734.08ms, mfu 4.01%
iter 8: loss 3.5956, time 31774.57ms, mfu 4.01%
iter 9: loss 3.7610, time 31749.35ms, mfu 4.01%
iter 10: loss 3.7035, time 31711.85ms, mfu 4.01%
iter 11: loss 3.5853, time 31625.33ms, mfu 4.01%
iter 12: loss 3.5244, time 31618.93ms, mfu 4.01%
iter 13: loss 3.5588, time 31592.44ms, mfu 4.02%
iter 14: loss 3.5339, time 31542.19ms, mfu 4.02%
iter 15: loss 3.6042, time 31568.39ms, mfu 4.02%
iter 16: loss 3.4638, time 31736.23ms, mfu 4.02%
iter 17: loss 3.4432, time 31840.00ms, mfu 4.02%
iter 18: loss 3.5116, time 31844.93ms, mfu 4.02%
iter 19: loss 3.4349

In [ ]:
# torch.save(model.state_dict(), "./out/backup_100_it.pt")

In [ ]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 100: train loss 2.9703, val loss 3.0115
saving checkpoint to out


In [44]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.0004475, rowwise=True)
print(pcnt_pruned)

0.05014696193897384


In [45]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [46]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 100: train loss 3.4429, val loss 3.4651
iter 100: loss 3.4515, time 134053.14ms, mfu -100.00%
iter 101: loss 3.3488, time 30309.06ms, mfu -100.00%
iter 102: loss 3.4385, time 30230.92ms, mfu -100.00%
iter 103: loss 3.2806, time 30344.04ms, mfu -100.00%
iter 104: loss 3.2395, time 30415.68ms, mfu -100.00%
iter 105: loss 3.2885, time 30490.47ms, mfu 4.18%
iter 106: loss 3.3089, time 30471.76ms, mfu 4.18%
iter 107: loss 3.2573, time 30379.12ms, mfu 4.18%
iter 108: loss 3.2136, time 30355.45ms, mfu 4.18%
iter 109: loss 3.3448, time 30293.81ms, mfu 4.18%
iter 110: loss 3.2640, time 30309.61ms, mfu 4.18%
iter 111: loss 3.2299, time 30361.85ms, mfu 4.18%
iter 112: loss 3.2516, time 30295.37ms, mfu 4.19%
iter 113: loss 3.3632, time 30268.49ms, mfu 4.19%
iter 114: loss 3.2608, time 30279.79ms, mfu 4.19%
iter 115: loss 3.2443, time 30227.85ms, mfu 4.19%
iter 116: loss 3.1991, time 30247.65ms, mfu 4.19%
iter 117: loss 3.2266, time 30359.05ms, mfu 4.19%
iter 118: loss 3.2023, time 30415.08ms,

In [47]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 150: train loss 3.0887, val loss 3.1267


In [48]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [49]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 150: train loss 3.0904, val loss 3.1263


In [50]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_150_pruned_0.05_direct.pt'))

saving checkpoint to out


In [5]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.000502, rowwise=True)
print(pcnt_pruned)

0.10086216140700122


In [6]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [7]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 150: train loss 8.1182, val loss 8.1177
iter 150: loss 8.1229, time 133027.23ms, mfu -100.00%
iter 151: loss 6.6373, time 30258.04ms, mfu -100.00%
iter 152: loss 5.4409, time 30234.30ms, mfu -100.00%
iter 153: loss 4.9134, time 30240.91ms, mfu -100.00%
iter 154: loss 4.5443, time 30438.89ms, mfu -100.00%
iter 155: loss 4.5381, time 30264.00ms, mfu 4.21%
iter 156: loss 4.4046, time 30239.01ms, mfu 4.21%
iter 157: loss 4.3113, time 30274.13ms, mfu 4.21%
iter 158: loss 4.2199, time 30290.47ms, mfu 4.21%
iter 159: loss 4.2449, time 30257.71ms, mfu 4.21%
iter 160: loss 4.1095, time 30416.80ms, mfu 4.20%
iter 161: loss 4.0877, time 30441.79ms, mfu 4.20%
iter 162: loss 3.9920, time 30482.72ms, mfu 4.20%
iter 163: loss 4.0831, time 30350.17ms, mfu 4.20%
iter 164: loss 3.9705, time 30256.57ms, mfu 4.20%
iter 165: loss 3.9368, time 30257.02ms, mfu 4.20%
iter 166: loss 3.8343, time 30396.94ms, mfu 4.20%
iter 167: loss 3.8741, time 30433.75ms, mfu 4.20%
iter 168: loss 3.8343, time 30341.58ms,

In [8]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 200: train loss 3.4852, val loss 3.5335


In [9]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [10]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 200: train loss 3.5085, val loss 3.5571


In [11]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_200_pruned_0.10_indirect.pt'))

saving checkpoint to out


In [5]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.0014, rowwise=True)
print(pcnt_pruned)

0.15003297353813358


In [6]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [7]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 200: train loss 10.7548, val loss 10.7539
iter 200: loss 10.7551, time 129212.54ms, mfu -100.00%
iter 201: loss 10.5756, time 29528.97ms, mfu -100.00%
iter 202: loss 10.3684, time 29294.47ms, mfu -100.00%
iter 203: loss 9.9969, time 29516.62ms, mfu -100.00%
iter 204: loss 9.9151, time 29553.52ms, mfu -100.00%
iter 205: loss 10.0143, time 29425.08ms, mfu 4.33%
iter 206: loss 9.9425, time 29320.66ms, mfu 4.33%
iter 207: loss 9.8947, time 29451.89ms, mfu 4.33%
iter 208: loss 9.8539, time 29470.36ms, mfu 4.33%
iter 209: loss 9.9455, time 29413.90ms, mfu 4.33%
iter 210: loss 9.9837, time 29352.33ms, mfu 4.33%
iter 211: loss 9.7774, time 29384.75ms, mfu 4.33%
iter 212: loss 9.8432, time 29434.38ms, mfu 4.33%
iter 213: loss 9.8641, time 29405.52ms, mfu 4.33%
iter 214: loss 9.9263, time 29366.07ms, mfu 4.33%
iter 215: loss 9.8600, time 29238.63ms, mfu 4.33%
iter 216: loss 9.7800, time 29300.09ms, mfu 4.33%
iter 217: loss 9.8844, time 29409.48ms, mfu 4.33%
iter 218: loss 9.8391, time 29446

In [8]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 250: train loss 9.4812, val loss 9.4494


In [9]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [10]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 250: train loss 9.7192, val loss 9.6854


In [11]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_250_pruned_0.15.pt'))

saving checkpoint to out


In [21]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.0016176, rowwise=True)
print(pcnt_pruned)

0.20002408482743725


In [22]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [23]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 250: train loss 10.8575, val loss 10.8572
iter 250: loss 10.8579, time 127536.73ms, mfu -100.00%
iter 251: loss 10.8251, time 28493.48ms, mfu -100.00%
iter 252: loss 10.8238, time 28655.35ms, mfu -100.00%
iter 253: loss 10.8246, time 28691.28ms, mfu -100.00%
iter 254: loss 10.8303, time 28491.23ms, mfu -100.00%
iter 255: loss 10.8268, time 28407.13ms, mfu 4.48%
iter 256: loss 10.8249, time 28289.82ms, mfu 4.48%
iter 257: loss 10.8250, time 28383.71ms, mfu 4.48%
iter 258: loss 10.8242, time 28450.57ms, mfu 4.48%
iter 259: loss 10.8236, time 28466.07ms, mfu 4.48%
iter 260: loss 10.8238, time 28373.42ms, mfu 4.48%
iter 261: loss 10.8261, time 28279.93ms, mfu 4.48%
iter 262: loss 10.8259, time 28317.83ms, mfu 4.49%
iter 263: loss 10.8237, time 28319.90ms, mfu 4.49%
iter 264: loss 10.8238, time 28239.58ms, mfu 4.49%
iter 265: loss 10.8242, time 28177.92ms, mfu 4.49%
iter 266: loss 10.8236, time 28155.33ms, mfu 4.50%
iter 267: loss 10.8235, time 28130.14ms, mfu 4.50%
iter 268: loss 10.8

In [24]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 300: train loss 10.7435, val loss 10.7443


In [25]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [26]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 300: train loss 10.8285, val loss 10.8281


In [27]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_300_pruned_0.20.pt'))

saving checkpoint to out


In [68]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.0076, rowwise=False)
print(pcnt_pruned)

0.2501452252399177


In [69]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [70]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 300: train loss 10.8288, val loss 10.8284
iter 300: loss 10.8280, time 125962.33ms, mfu -100.00%
iter 301: loss 10.8191, time 28281.72ms, mfu -100.00%
iter 302: loss 10.8145, time 28294.12ms, mfu -100.00%
iter 303: loss 10.8107, time 28526.59ms, mfu -100.00%
iter 304: loss 10.7951, time 28505.07ms, mfu -100.00%
iter 305: loss 10.7849, time 28582.73ms, mfu 4.45%
iter 306: loss 10.7583, time 28655.99ms, mfu 4.45%
iter 307: loss 10.7331, time 28759.91ms, mfu 4.45%
iter 308: loss 10.6993, time 28825.21ms, mfu 4.45%
iter 309: loss 10.6734, time 28968.24ms, mfu 4.44%
iter 310: loss 10.6556, time 29154.32ms, mfu 4.43%
iter 311: loss 10.6779, time 29203.73ms, mfu 4.43%
iter 312: loss 10.6679, time 29310.14ms, mfu 4.42%
iter 313: loss 10.6331, time 29101.28ms, mfu 4.41%
iter 314: loss 10.6713, time 28990.01ms, mfu 4.41%
iter 315: loss 10.6455, time 29074.23ms, mfu 4.41%
iter 316: loss 10.6510, time 29103.85ms, mfu 4.41%
iter 317: loss 10.6647, time 29185.71ms, mfu 4.40%
iter 318: loss 10.6

In [71]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 350: train loss 10.5773, val loss 10.5797


In [72]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [73]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 350: train loss 10.8228, val loss 10.8234


In [74]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_350_pruned_0.25.pt'))

saving checkpoint to out


In [84]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.0155, rowwise=False)
print(pcnt_pruned)

0.30162594039540297


In [85]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [86]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 350: train loss 10.8228, val loss 10.8236
iter 350: loss 10.8227, time 126484.99ms, mfu -100.00%
iter 351: loss 10.8241, time 28681.25ms, mfu -100.00%
iter 352: loss 10.8136, time 28540.77ms, mfu -100.00%
iter 353: loss 10.7949, time 28549.83ms, mfu -100.00%
iter 354: loss 10.7921, time 28625.11ms, mfu -100.00%
iter 355: loss 10.7650, time 28681.81ms, mfu 4.44%
iter 356: loss 10.7464, time 28663.46ms, mfu 4.44%
iter 357: loss 10.7325, time 28790.82ms, mfu 4.44%
iter 358: loss 10.6689, time 28926.73ms, mfu 4.43%
iter 359: loss 10.6428, time 29058.49ms, mfu 4.43%
iter 360: loss 10.6708, time 29201.52ms, mfu 4.42%
iter 361: loss 10.6539, time 29322.24ms, mfu 4.41%
iter 362: loss 10.6985, time 29414.35ms, mfu 4.41%
iter 363: loss 10.6260, time 29199.23ms, mfu 4.40%
iter 364: loss 10.6780, time 29029.11ms, mfu 4.40%
iter 365: loss 10.6373, time 29137.85ms, mfu 4.40%
iter 366: loss 10.5978, time 29243.21ms, mfu 4.39%
iter 367: loss 10.6331, time 29191.38ms, mfu 4.39%
iter 368: loss 10.6

In [87]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 400: train loss 10.5507, val loss 10.5547


In [88]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [89]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 400: train loss 10.8535, val loss 10.8558


In [90]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_400_pruned_0.30.pt'))

saving checkpoint to out


In [112]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.0235, rowwise=False)
print(pcnt_pruned)

0.3528153288068805


In [113]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [114]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 400: train loss 10.8857, val loss 10.8884
iter 400: loss 10.8839, time 126673.64ms, mfu -100.00%
iter 401: loss 10.8244, time 28614.73ms, mfu -100.00%
iter 402: loss 10.8067, time 28438.76ms, mfu -100.00%
iter 403: loss 10.7972, time 28673.97ms, mfu -100.00%
iter 404: loss 10.7793, time 28749.23ms, mfu -100.00%
iter 405: loss 10.7555, time 28774.50ms, mfu 4.42%
iter 406: loss 10.7335, time 28790.37ms, mfu 4.42%
iter 407: loss 10.6860, time 28884.22ms, mfu 4.42%
iter 408: loss 10.6607, time 28843.65ms, mfu 4.42%
iter 409: loss 10.6170, time 28922.03ms, mfu 4.42%
iter 410: loss 10.6314, time 28964.27ms, mfu 4.42%
iter 411: loss 10.6249, time 28990.36ms, mfu 4.42%
iter 412: loss 10.5904, time 28957.18ms, mfu 4.41%
iter 413: loss 10.6219, time 28950.46ms, mfu 4.41%
iter 414: loss 10.5691, time 28974.24ms, mfu 4.41%
iter 415: loss 10.5573, time 29030.50ms, mfu 4.41%
iter 416: loss 10.5602, time 29048.54ms, mfu 4.41%
iter 417: loss 10.5578, time 29040.70ms, mfu 4.40%
iter 418: loss 10.5

In [115]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 450: train loss 10.5238, val loss 10.5269


In [116]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [117]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 450: train loss 10.9864, val loss 10.9887


In [118]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_450_pruned_0.35.pt'))

saving checkpoint to out


In [135]:
with torch.no_grad():
  pcnt_pruned = model.prune(m=0.165, rowwise=False)
print(pcnt_pruned)

0.9079216491612829


In [136]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [137]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

step 450: train loss 10.8343, val loss 10.8335
iter 450: loss 10.8342, time 115888.02ms, mfu -100.00%
iter 451: loss 10.8303, time 26731.39ms, mfu -100.00%
iter 452: loss 10.8269, time 26477.96ms, mfu -100.00%
iter 453: loss 10.8224, time 26738.61ms, mfu -100.00%
iter 454: loss 10.8164, time 26810.90ms, mfu -100.00%
iter 455: loss 10.8132, time 26749.35ms, mfu 4.76%
iter 456: loss 10.8090, time 26686.05ms, mfu 4.76%
iter 457: loss 10.8165, time 26652.46ms, mfu 4.76%
iter 458: loss 10.8158, time 26678.87ms, mfu 4.76%
iter 459: loss 10.8254, time 26745.39ms, mfu 4.76%
iter 460: loss 10.8079, time 26800.73ms, mfu 4.76%
iter 461: loss 10.8194, time 26826.99ms, mfu 4.76%
iter 462: loss 10.8139, time 26819.42ms, mfu 4.76%
iter 463: loss 10.8221, time 26702.82ms, mfu 4.76%
iter 464: loss 10.8134, time 26613.85ms, mfu 4.76%
iter 465: loss 10.8108, time 26531.13ms, mfu 4.77%
iter 466: loss 10.8041, time 26509.79ms, mfu 4.77%
iter 467: loss 10.8164, time 26606.92ms, mfu 4.77%
iter 468: loss 10.8

In [138]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 500: train loss 10.8073, val loss 10.8089


In [139]:
with torch.no_grad():
  for module in model.modules():
    if isinstance(module, PrunableLinear):
      module.weight *= module.mask

In [140]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss, eval_only=True)

step 500: train loss 10.8088, val loss 10.8105


In [141]:
print(f"saving checkpoint to {out_dir}")
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'model_args': model_args,
    'iter_num': iter_num,
    'best_val_loss': best_val_loss,
    'config': config,
}, os.path.join(out_dir, 'ckpt_500_pruned_0.91.pt'))

saving checkpoint to out


In [6]:
from pruning import compress_layers, compress

In [7]:
compress_layers(model, device=device_type)
torch.cuda.empty_cache()
import gc; gc.collect()

94

In [8]:
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): CompressedLinear(in_features=1024, out_features=3072, bias=True)
          (c_proj): CompressedLinear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): CompressedLinear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): CompressedLinear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): CompressedLinear(in_features=1024, out_features=50257, bias=False)
)

In [9]:
torch.save(model, 'test.pt')

In [5]:
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): CompressedLinear(in_features=1024, out_features=3072, bias=True)
          (c_proj): CompressedLinear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): CompressedLinear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): CompressedLinear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): CompressedLinear(in_features=1024, out_features=50257, bias=False)
)

In [16]:
iter_num, best_val_loss = finetune(model, iter_num, best_val_loss)

AttributeError: ignored

In [11]:
from pruning import CompressedLinear

In [13]:
for name, module in model.named_children():
    if len(list(module.children())) > 0:
        compress_layers(module, device)  # compound module: recurse
    if isinstance(module, CompressedLinear):
        print(name)
        print(module.rows_not_pruned)

lm_head
tensor([True, True, True,  ..., True, True, True], device='cuda:0')


In [8]:
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-23): 24 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): CompressedLinear(in_features=1024, out_features=3072, bias=True)
          (c_proj): CompressedLinear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): CompressedLinear(in_features=1024, out_features=4096, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): CompressedLinear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): CompressedLinear(in_features=1024, out_features=50257, bias=False)
)